In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/all-metrics/All_metrics.xlsx


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_excel(r"/kaggle/input/all-metrics/All_metrics.xlsx")
df.rename(columns={"hist_player_score": "hist_player_performance"}, inplace=True)
df.index = df.game_id
df

,game_id,first,time_control_name,game_end_reason,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,Bot_nickname,Player_nickname,Bot_turns_count,Player_turns_count,Bot_max_point,Player_max_point,Bot_min_point,Player_min_point,Bot_avg_bingo,Player_avg_bingo,Bot_word_len,Player_word_len,Bot_count_letters_JQXZ,Player_count_letters_JQXZ,Bot_rack_used_rate,Player_rack_used_rate,Bot_win,Player_win,Bot_game,Player_game,Bot_score,Player_score,Bot_rating,Player_rating,created_at,hist_player_performance,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter,hist_player_score_20,last_game_win_flag,Player_max_point_player_wise,Player_avg_bingo_player_wise,Player_word_len_player_wise,Player_score_player_wise,hist_player_score_player_wise,Player_Double_Word_player_wise,Player_Triple_Word_player_wise,Player_Double_Letter_player_wise,Player_Triple_Letter_player_wise,hist_player_RATING_20_player_wise
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43828,43828,HastyBot,ultrablitz,RESIGNED,CSW21,15,0,RATED,1,61.405180,HastyBot,0188889876,16,16,72,14,0,-10,0.062500,0.000000,4.187500,0.312500,2,0,0.638095,0.178571,1,0,14112,1,472,15,2148,1979.0,2022-09-11 13:06:11,1700.0,4,5,8,4,0,0,1,1,1700.00,0,14,0.000000,0.312500,15,1700.0,0.000000,0.000000,1.000000,1.000000,1700.00
5765,5765,0927399533,regular,TIME,CSW21,1200,0,RATED,1,1268.252752,BetterBot,0927399533,11,12,92,35,16,-10,0.272727,0.000000,4.090909,2.250000,4,0,0.584416,0.482143,1,0,4847,1,462,141,2012,1447.0,2022-08-09 14:25:28,1700.0,2,3,7,5,4,2,3,0,1700.00,0,35,0.026824,2.267920,141,1700.0,4.315789,3.157895,5.052632,2.631579,1700.00
40264,40264,BetterBot,regular,TIME,CSW21,1200,0,RATED,1,1262.786704,BetterBot,0927399533,12,12,74,36,8,-10,0.166667,0.000000,4.166667,3.000000,0,2,0.595238,0.467532,1,0,4871,2,356,212,1966,1408.0,2022-08-09 15:26:54,1691.2,7,0,4,6,3,4,5,5,1695.60,0,36,0.026824,2.267920,212,1691.2,4.315789,3.157895,5.052632,2.631579,1695.60
44376,44376,0927399533,regular,STANDARD,CSW21,1200,0,RATED,1,1209.746654,BetterBot,0927399533,15,16,102,75,8,-10,0.125000,0.062500,3.750000,2.437500,4,0,0.625397,0.396825,1,0,5327,3,535,227,2090,1372.0,2022-08-10 16:28:43,1677.8,6,4,9,5,4,3,9,0,1688.90,0,75,0.026824,2.267920,227,1677.8,4.315789,3.157895,5.052632,2.631579,1688.90
32633,32633,BetterBot,regular,STANDARD,CSW21,1200,0,RATED,1,1231.166683,BetterBot,0927399533,15,16,85,85,6,-10,0.066667,0.117647,3.400000,2.823529,3,1,0.520952,0.519048,0,1,5336,4,383,391,2114,1367.0,2022-08-10 16:49:00,1676.8,7,1,6,5,6,5,4,2,1688.40,0,85,0.026824,2.267920,391,1676.8,4.315789,3.157895,5.052632,2.631579,1688.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7264,7264,zuzu,regular,STANDARD,CSW21,1200,0,RATED,1,683.258274,HastyBot,zuzu,13,14,77,78,10,0,0.076923,0.066667,3.846154,3.200000,3,1,0.549451,0.538462,1,0,15306,7,414,327,2231,1855.0,2022-09-13 09:23:17,1750.7,5,2,5,4,5,5,4,1,1725.35,0,78,0.086263,3.488829,327,1750.7,5.000000,3.428571,4.142857,2.428571,1725.35
66022,66022,zyzzyva,regular,STANDARD,NWL20,1380,0,RATED,1,626.274697,STEEBot,zyzzyva,13,13,78,66,0,0,0.142857,0.076923,3.285714,3.846154,3,1,0.649351,0.595238,1,0,573,1,393,357,1859,1641.0,2022-07-28 18:47:59,1700.0,5,1,5,2,6,3,4,1,1700.00,0,66,0.115876,3.690110,357,1700.0,5.750000,3.250000,5.250000,3.250000,1700.00
2581,2581,STEEBot,regular,STANDARD,NWL20,1380,0,RATED,1,775.765387,STEEBot,zyzzyva,12,11,70,76,13,17,0.076923,0.181818,3.923077,4.000000,3,1,0.642857,0.571429,0,1,579,2,358,438,1863,1637.0,2022-07-28 18:58:36,1675.9,5,2,6,5,3,4,6,5,1687.95,0,76,0.115876,3.690110,438,1675.9,5.750000,3.250000,5.250000,3.250000,1687.95


In [3]:
df.dtypes

game_id                                int64
first                                 object
time_control_name                     object
game_end_reason                       object
lexicon                               object
initial_time_seconds                   int64
increment_seconds                      int64
rating_mode                           object
max_overtime_minutes                   int64
game_duration_seconds                float64
Bot_nickname                          object
Player_nickname                       object
Bot_turns_count                        int64
Player_turns_count                     int64
Bot_max_point                          int64
Player_max_point                       int64
Bot_min_point                          int64
Player_min_point                       int64
Bot_avg_bingo                        float64
Player_avg_bingo                     float64
Bot_word_len                         float64
Player_word_len                      float64
Bot_count_

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Drop 'game_id'
df = df.drop('game_id', axis=1)

# Set binary column based on 'first' values
df['first_bot'] = df['first'].isin(["HastyBot", "BetterBot", "STEEBot"]).astype(int)
df = df.drop('first', axis=1)

# One-hot encode categorical columns
categorical_columns = ['time_control_name', 'game_end_reason', 'lexicon', 'rating_mode', 'Bot_nickname']
df = pd.get_dummies(df, columns=categorical_columns)

# Drop 'Player_nickname' and 'created_at'
df = df.drop(['Player_nickname', 'created_at', "Bot_game"], axis=1)

# Move 'Player_rating' to the last column
cols = [col for col in df.columns if col != 'Player_rating'] + ['Player_rating']
df = df[cols]

df

,initial_time_seconds,increment_seconds,max_overtime_minutes,game_duration_seconds,Bot_turns_count,Player_turns_count,Bot_max_point,Player_max_point,Bot_min_point,Player_min_point,Bot_avg_bingo,Player_avg_bingo,Bot_word_len,Player_word_len,Bot_count_letters_JQXZ,Player_count_letters_JQXZ,Bot_rack_used_rate,Player_rack_used_rate,Bot_win,Player_win,Player_game,Bot_score,Player_score,Bot_rating,hist_player_performance,Bot_Double_Word,Bot_Triple_Word,Bot_Double_Letter,Bot_Triple_Letter,Player_Double_Word,Player_Triple_Word,Player_Double_Letter,Player_Triple_Letter,hist_player_score_20,last_game_win_flag,Player_max_point_player_wise,Player_avg_bingo_player_wise,Player_word_len_player_wise,Player_score_player_wise,hist_player_score_player_wise,Player_Double_Word_player_wise,Player_Triple_Word_player_wise,Player_Double_Letter_player_wise,Player_Triple_Letter_player_wise,hist_player_RATING_20_player_wise,first_bot,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,time_control_name_ultrablitz,game_end_reason_CONSECUTIVE_ZEROES,game_end_reason_RESIGNED,game_end_reason_STANDARD,game_end_reason_TIME,lexicon_CSW21,lexicon_ECWL,lexicon_NSWL20,lexicon_NWL20,rating_mode_CASUAL,rating_mode_RATED,Bot_nickname_BetterBot,Bot_nickname_HastyBot,Bot_nickname_STEEBot,Player_rating
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
43828,15,0,1,61.405180,16,16,72,14,0,-10,0.062500,0.000000,4.187500,0.312500,2,0,0.638095,0.178571,1,0,1,472,15,2148,1700.0,4,5,8,4,0,0,1,1,1700.00,0,14,0.000000,0.312500,15,1700.0,0.000000,0.000000,1.000000,1.000000,1700.00,1,False,False,False,True,False,True,False,False,True,False,False,False,False,True,False,True,False,1979.0
5765,1200,0,1,1268.252752,11,12,92,35,16,-10,0.272727,0.000000,4.090909,2.250000,4,0,0.584416,0.482143,1,0,1,462,141,2012,1700.0,2,3,7,5,4,2,3,0,1700.00,0,35,0.026824,2.267920,141,1700.0,4.315789,3.157895,5.052632,2.631579,1700.00,0,False,False,True,False,False,False,False,True,True,False,False,False,False,True,True,False,False,1447.0
40264,1200,0,1,1262.786704,12,12,74,36,8,-10,0.166667,0.000000,4.166667,3.000000,0,2,0.595238,0.467532,1,0,2,356,212,1966,1691.2,7,0,4,6,3,4,5,5,1695.60,0,36,0.026824,2.267920,212,1691.2,4.315789,3.157895,5.052632,2.631579,1695.60,1,False,False,True,False,False,False,False,True,True,False,False,False,False,True,True,False,False,1408.0
44376,1200,0,1,1209.746654,15,16,102,75,8,-10,0.125000,0.062500,3.750000,2.437500,4,0,0.625397,0.396825,1,0,3,535,227,2090,1677.8,6,4,9,5,4,3,9,0,1688.90,0,75,0.026824,2.267920,227,1677.8,4.315789,3.157895,5.052632,2.631579,1688.90,0,False,False,True,False,False,False,True,False,True,False,False,False,False,True,True,False,False,1372.0
32633,1200,0,1,1231.166683,15,16,85,85,6,-10,0.066667,0.117647,3.400000,2.823529,3,1,0.520952,0.519048,0,1,4,383,391,2114,1676.8,7,1,6,5,6,5,4,2,1688.40,0,85,0.026824,2.267920,391,1676.8,4.315789,3.157895,5.052632,2.631579,1688.40,1,False,False,True,False,False,False,True,False,True,False,False,False,False,True,True,False,False,1367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7264,1200,0,1,683.258274,13,14,77,78,10,0,0.076923,0.066667,3.846154,3.200000,3,1,0.549451,0.538462,1,0,7,414,327,2231,1750.7,5,2,5,4,5,5,4,1,1725.35,0,78,0.086263,3.488829,327,1750.7,5.000000,3.428571,4.142857,2.428571,1725.35,0,False,False,True,False,False,False,True,False,True,False,False,False,False,True,False,True,False,1855.0
66022,1380,0,1,626.274697,13,13,78,66,0,0,0.142857,0.076923,3.285714,3.846154,3,1,0.649351,0.595238,1,0,1,393,357,1859,1700.0,5,1,5,2,6,3,4,1,1700.00,0,66,0.115876,3.690110,357,1700.0,5.750000,3.250000,5.250000,3.250000,1700.00,0,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,True,1641.0
2581,1380,0,1,775.

In [5]:
train = df[df["Player_rating"].isnull()==False]
test = df[df["Player_rating"].isnull()==True]

In [6]:
df = train

In [7]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Assuming df is your DataFrame
X = df.drop('Player_rating', axis=1)  # Features
y = df['Player_rating']  # Target variable

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)

# Fit the model to the data
rf_model.fit(X, y)

# Get feature importances
feature_importances = rf_model.feature_importances_

# Create a DataFrame to store the results
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the sorted DataFrame
print(feature_importance_df)

                               Feature    Importance
24             hist_player_performance  2.712390e-01
39       hist_player_score_player_wise  1.933953e-01
37         Player_word_len_player_wise  1.269981e-01
36        Player_avg_bingo_player_wise  7.401684e-02
40      Player_Double_Word_player_wise  4.422883e-02
..                                 ...           ...
51            game_end_reason_RESIGNED  3.836491e-05
52            game_end_reason_STANDARD  3.744009e-05
53                game_end_reason_TIME  3.555620e-05
50  game_end_reason_CONSECUTIVE_ZEROES  5.164689e-06
56                      lexicon_NSWL20  7.626653e-07

[63 rows x 2 columns]


In [8]:
selected_features = feature_importance_df[feature_importance_df['Importance'] > 0.005]

# Sort the DataFrame by importance in descending order
selected_features = selected_features.sort_values(by='Importance', ascending=False)

# Print the sorted DataFrame
selected_features

,Feature,Importance
24,hist_player_performance,0.271239
39,hist_player_score_player_wise,0.193395
37,Player_word_len_player_wise,0.126998
36,Player_avg_bingo_player_wise,0.074017
40,Player_Double_Word_player_wise,0.044229
33,hist_player_score_20,0.033611
44,hist_player_RATING_20_player_wise,0.031552
42,Player_Double_Letter_player_wise,0.029270
54,lexicon_CSW21,0.027913
41,Player_Triple_Word_player_wise,0.026939


In [9]:
# Assuming df is your original DataFrame
selected_feature_names = selected_features['Feature'].tolist()

# Select columns with the important features
df_selected = df[selected_feature_names + ['Player_rating']]

# Display the selected DataFrame
df_selected.head()

,hist_player_performance,hist_player_score_player_wise,Player_word_len_player_wise,Player_avg_bingo_player_wise,Player_Double_Word_player_wise,hist_player_score_20,hist_player_RATING_20_player_wise,Player_Double_Letter_player_wise,lexicon_CSW21,Player_Triple_Word_player_wise,Player_Triple_Letter_player_wise,lexicon_NWL20,Bot_rating,Player_game,initial_time_seconds,rating_mode_RATED,rating_mode_CASUAL,game_duration_seconds,Player_rating
game_id,,,,,,,,,,,,,,,,,,,
43828,1700.0,1700.0,0.31250,0.000000,0.000000,1700.0,1700.0,1.000000,True,0.000000,1.000000,False,2148,1,15,True,False,61.405180,1979.0
5765,1700.0,1700.0,2.26792,0.026824,4.315789,1700.0,1700.0,5.052632,True,3.157895,2.631579,False,2012,1,1200,True,False,1268.252752,1447.0
40264,1691.2,1691.2,2.26792,0.026824,4.315789,1695.6,1695.6,5.052632,True,3.157895,2.631579,False,1966,2,1200,True,False,1262.786704,1408.0
44376,1677.8,1677.8,2.26792,0.026824,4.315789,1688.9,1688.9,5.052632,True,3.157895,2.631579,False,2090,3,1200,True,False,1209.746654,1372.0
32633,1676.8,1676.8,2.26792,0.026824,4.315789,1688.4,1688.4,5.052632,True,3.157895,2.631579,False,2114,4,1200,True,False,1231.166683,1367.0


In [10]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

from sklearn.tree import DecisionTreeRegressor

# Assuming df_selected is your DataFrame
X = df_selected.drop('Player_rating', axis=1)
y = df_selected['Player_rating']

# Create a pipeline with DecisionTreeRegressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Decision trees don't require scaling, but keeping it for consistency
    ('dt', DecisionTreeRegressor())
])

# Define parameter grid for grid search
param_grid = {
    'dt__max_depth': [None, 6, 8, 12, 15],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4, 8]
}

# Define RMSE as the scoring metric for GridSearchCV
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(np.mean((y_true - y_pred)**2)))

# Initialize GridSearchCV with verbose set to a positive integer
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring=rmse_scorer, n_jobs=-1, verbose=2)

# Fit the model using partial_fit to print live progress
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_
print("\nBest Model:", best_model)

# Get cross-validation scores
print("\nRunning cross-validation. This may take some time...")
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # Specify the same number of splits as the GridSearchCV
fold = 1
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    fold_rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    print(f"Fold {fold}: RMSE = {fold_rmse}")
    fold += 1


Fitting 10 folds for each of 60 candidates, totalling 600 fits
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=2; total time=   1.1s
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=2; total time=   1.1s
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=2; total time=   1.1s
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=5; total time=   1.0s
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=5; total time=   1.0s
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=10; total time=   1.0s
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=10; total time=   1.0s
[CV] END dt__max_depth=None, dt__min_samples_leaf=1, dt__min_samples_split=10; total time=   1.0s
[CV] END dt__max_depth=None, dt__min_samples_leaf=2, dt__min_samples_split=2; total time=   1.0s
[CV] END dt__max_depth=None, dt__min_samples_leaf=2, dt__min_

In [11]:
best_model

Pipeline(steps=[('scaler', StandardScaler()), ('dt', DecisionTreeRegressor())])

In [12]:
test_df_selected = test[selected_feature_names + ['Player_rating']]

# Display the selected DataFrame
test_df_selected.head()
# Assuming df_selected is your DataFrame
test_X = test_df_selected.drop('Player_rating', axis=1)
test_y = test_df_selected['Player_rating']

test_X["Predictions"] = best_model.predict(test_X)
test_X["game_id"] = test_X.index
test_X

,hist_player_performance,hist_player_score_player_wise,Player_word_len_player_wise,Player_avg_bingo_player_wise,Player_Double_Word_player_wise,hist_player_score_20,hist_player_RATING_20_player_wise,Player_Double_Letter_player_wise,lexicon_CSW21,Player_Triple_Word_player_wise,Player_Triple_Letter_player_wise,lexicon_NWL20,Bot_rating,Player_game,initial_time_seconds,rating_mode_RATED,rating_mode_CASUAL,game_duration_seconds,Predictions,game_id
game_id,,,,,,,,,,,,,,,,,,,,
51857,1700.0,1700.0,2.785714,0.071429,6.000000,1700.00,1700.00,3.000000,True,4.000000,1.000000,False,1990,1,1200,True,False,632.083319,1592.0,51857
31730,1700.0,1700.0,1.703782,0.000000,3.500000,1700.00,1700.00,4.000000,True,2.000000,3.500000,False,2269,1,1200,True,False,256.453327,1481.0,31730
50150,1716.9,1716.9,1.703782,0.000000,3.500000,1708.45,1708.45,4.000000,True,2.000000,3.500000,False,2220,2,1200,True,False,245.164982,1481.0,50150
18710,1700.0,1700.0,3.135054,0.070248,3.909091,1700.00,1700.00,5.636364,True,2.909091,2.590909,False,2065,1,780,False,True,798.728617,1787.0,18710
46342,1696.5,1696.5,3.135054,0.070248,3.909091,1698.25,1698.25,5.636364,True,2.909091,2.590909,False,2065,2,780,False,True,841.983634,1787.0,46342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5506,2133.3,2133.3,3.743269,0.176743,4.520833,2079.25,2079.25,5.395833,True,2.937500,2.895833,False,2218,44,900,True,False,471.077622,2051.0,5506
26454,2056.9,2056.9,3.743269,0.176743,4.520833,2083.65,2083.65,5.395833,True,2.937500,2.895833,False,2222,45,900,True,False,421.667981,2032.0,26454
14102,2060.3,2060.3,3.743269,0.176743,4.520833,2079.50,2079.50,5.395833,True,2.937500,2.895833,False,2235,46,900,True,False,295.437074,2032.0,14102


In [13]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
file_name = f"final_test_results_{timestamp}.csv"



final_test_results = test_X[["game_id", "Predictions"]]
final_test_results.rename(columns={"Predictions": "rating"}, inplace=True)
final_test_results_reset = final_test_results.reset_index(drop=True)
final_test_results_sorted = final_test_results_reset.sort_values(by='game_id', ascending=True)
final_test_results_sorted.to_csv(file_name, index=False)


/tmp/ipykernel_47/1987286698.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_test_results.rename(columns={"Predictions": "rating"}, inplace=True)
